In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 31: Likelihood Ratio Tests

Recall maximum likelihood estimators. These are obtained by maximizing the likelihood function with respect to $\theta$, the parameter of interest. Let's go through an example:

### Example 1: Poisson Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the Poisson distribution with unknown parameter $\lambda$. Find $\hat{\lambda}_{ML}$, the maximum likelihood estimate of $\lambda$. 

Since the poisson distribution paramater $\lambda$ represents the number of occurences of an event within a certain time interval. 

$$
L(\lambda\mid \textbf{x}) = \prod_{i=1}^n f(x_i;\theta) = \prod_{i=1}^n {\lambda^{ X_i} e^{-\lambda} \over X_i !} = {\lambda^{\sum X_i} e^{-\lambda n} \over \prod_{i=1}^n X_i !}
$$

$$
l(\lambda \mid \textbf{x})=log(L(\lambda \mid \textbf{x}))=log({\lambda^{\sum X_i} e^{-\lambda n} \over \sum_{i=1}^n X_i ! })=(\sum X_i ) log(\lambda) -\lambda n - \sum_{i=1}^n log(X_i !) 
$$

Set the derivative of $l(\lambda \mid \textbf{x})$ to 0:
$$
0={d \over d \lambda}[l(\lambda \mid \textbf{x})]= {\sum_{i=1}^n X_i \over \lambda}-n
$$

$$
n\lambda=\sum_{i=1}^n X_i
$$
$$
\hat{\lambda}_{ML}={\sum_{i=1}^n X_i \over n}
$$

### Likelihood Ratio Tests

Assume you are testing a hypothesis:
$$
H_0: \theta=\theta_0
$$
$$
H_1: \theta\neq \theta_0
$$

The idea behind a likelihood ratio test is to compare the likelihood of the hypothesized value ($L(\theta_0\mid\textbf{x})$) to the maximum likelihood given the data ($L(\hat{\theta}_{ML}\mid\textbf{x})$). If the hypothesized value of $\theta$ were feasible, the likelihood under $\theta_0$ should be close to the max. If the hypothesized value of $\theta$ were not feasible, $L(\theta_0\mid\textbf{x})$ should be much smaller. To make the comparison, we consider the likelihood ratio test statistic, $\Lambda$: 

$$
\Lambda=\frac{L(\theta_0\mid\textbf{x})}{L(\hat{\theta}_{ML}\mid\textbf{x})}
$$

Because $\hat{\theta}_{ML}$ is the maximum likelihood estimator, this ratio has a maximum value of 1. Large values of $\Lambda$ (close to 1) indicate that $\theta_0$ is feasible (lack of evidence to reject). Small values of $\Lambda$ (close to 0) indicate $\theta_0$ is not feasible (evidence to reject). 

But how close to 0 is "close"? 

To evaluate this, we will take advantage of a helpful result. It turns out that if the null hypothesis were true, $-2\log \Lambda$ approximately follows the chi-squared distribution with 1 degree of freedom. The proof is outside the scope of this class. 

[We have not yet talked about the chi-squared distribution. To learn more, consult scipy help (`scipy.stats.chi2`). This distribution has one parameter that we care about: degrees of freedom, referenced in scipy as `df`. Bottom line, a random variable that has a chi-squared distribution with `df` degrees of freedom has a domain of $[0,\infty)$ and an expected value of `df`.]

$$
-2\log \Lambda = 2\left[l(\hat{\theta}_{ML}\mid \textbf{x})-l(\theta_0\mid \textbf{x})\right] \approx \textsf{Chisq}(1)
$$

It is a little bit more intuitive to consider $-2\log \Lambda$; large values of this test statistic indicate evidence to reject the null, which is consistent with most other hypothesis tests we've looked at. 

### Example 2: Likelihood ratio test on $\lambda$ from Poisson distribution

(Example taken from Pruim 2011). An instructor believes the the number of students who arrive late for class follows a Poisson process with an average of 1 late arrival per lesson. In 10 consecutive lessons, he found the following number of late arrivals: (1,1,0,4,2,1,3,0,0,2). Conduct a likelihood ratio test on the following hypothesis:

$$
H_0: \lambda = 1
$$
$$
H_1: \lambda \neq 1
$$

First we find our $ \hat{\lambda}_{ML}$ based on our data: 

In [2]:
act_data=[1, 1, 0, 4, 2, 1, 3, 0, 0, 2]
lam_ML=np.mean(act_data)
lam_ML

1.4

$$
L(\lambda\mid \textbf{x}) = {\lambda^{\sum X_i} e^{-\lambda n} \over \prod_{i=1}^n X_i !}
$$

Then we can find $\Lambda$:

In [8]:
hyp_lam=1 #what we are testing
n=10 #trials
like_denom=1
for i in act_data:
    like_denom=like_denom*factorial(act_data[i])
    
hyp_like=(hyp_lam**(sum(act_data))*e**(-1*hyp_lam*n))/(like_denom)

ML_like=(lam_ML**(sum(act_data))*e**(-1*lam_ML*n))/(like_denom)
LRTS=hyp_like/ML_like
chi1=-2*log(LRTS)
print('LRTS:', LRTS, 'Chi(1) value:', chi1)
crit=stats.chi2.ppf(.95, 1)
print('Using a 5% confidence, our critical chi squared value is:', crit)

LRTS: 0.49134374096645217 Chi(1) value: 1.4212226253939613
Using a 5% confidence, our critical chi squared value is: 3.841458820694124


Since our chi squared value of 1.42 is less than our critical chi squared value of 3.84, we fail to reject the null hypothesis.

#### Alternative Method

In this method, I simply use the sample mean as the test statistic. The observed value was 1.4. In the below, I simulate under the hypothesized $\lambda$, 1, and determine how often the sample mean was further away from 1 as was 1.4. 

In [11]:
ts2=[np.mean(stats.poisson.rvs(1,size=10)) for _ in np.arange(10000)]
np.mean(ts2>=np.mean(act_data))+np.mean(ts2<=(1-(np.mean(act_data)-1)))

0.2727